In [1]:
import torch
import torch.nn as nn

In [ ]:
class PerWordTransformer(nn.Module):
    def __init__(self, max_tokens, embedding_size, num_heads, num_encoder_layers):
        super(nn.Module, self).__init__()
        self.transformer = nn.Transformer(d_model=embedding_size*max_tokens, nhead=num_heads, num_encoder_layers=num_encoder_layers)
        self.classification_head = nn.Linear(embedding_size*max_tokens, 2)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.transformer(x)
        x = self.classification_head(x)
        x = self.softmax(x)
        return x
        

In [2]:
max_tokens = 128
emb_size = 512
transformer = nn.Transformer(d_model=128, nhead=16, num_encoder_layers=12)

c:\Users\lucas\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
